In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [3]:
from crewai import Agent, Task, Crew

In [5]:
import os
import sys

sys.path.append('/Users/kithmalgunathillake/desktop/crewAI')

from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [7]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [12]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [13]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [14]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate tasks to the right team members and ask the right questions to get the best insights on monitoring and analyzing market data for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL",
"context": "The data includes historical stock prices, trading volume, news sentiment, and any other relevant factors that may impact AAPL's market performance. Use statistical modeling and machine learning techniques to identify trends and predict market movements."
}

> Entering new CrewAgentExecutor chain...
I need to gather real-time data on AAPL's market performance to identify trends and predict movements.

Action

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Trading Strategies for AAPL:

Introduction:
The following risk analysis report provides a comprehensive evaluation of the potential risks associated with the proposed trading strategies for AAPL, along with recommendations for risk mitigation. The analysis is based on historical stock prices, market trends, and significant price fluctuations for AAPL.

Risk Assessment:
1. Market Risk:
- AAPL is subject to market volatility, influenced by factors such as economic indicators, geopolitical events, and industry trends.
- Mitigation Strategy: Implement stop-loss orders and regularly monitor market trends to adjust trading strategies accordingly.

2. Price Fluctuation Risk:
- AAPL has experienced significant price fluctuations over the years, with both all-time high and low prices.
- Mitigation Strategy: Diversify the investment portfolio to reduce concentration risk and utilize hedging strategies to mitigate potential downside risks.

3. Trend-Following Strategy Risk:
- Implementing a trend-following strategy may expose the trading activities to risks associated with market trends and price movements.
- Mitigation Strategy: Continuously monitor market trends and adjust execution plans accordingly to capitalize on growth opportunities while managing risks effectively.

4. Execution Plan Risk:
- The execution plans should align with historical stock price movements and market trends to make informed trading decisions.
- Mitigation Strategy: Regularly review and update execution plans based on the latest market information to achieve the user's investment goals.

Recommendations:
1. Diversification: Consider diversifying the investment portfolio to reduce concentration risk associated with AAPL stock.
2. Stop-loss Orders: Implement stop-loss orders to automatically sell AAPL stock if prices drop below a certain threshold to limit potential losses.
3. Hedging: Utilize options or futures contracts to hedge against potential downside risks in AAPL stock.
4. Regular Monitoring: Continuously monitor market trends, news, and financial reports related to AAPL to stay informed and react promptly to any changes.
5. Consultation: Seek advice from financial experts or advisors to gain additional insights and make informed decisions regarding trading activities involving AAPL.

By implementing these risk mitigation strategies, the trading activities for AAPL can align more effectively with the firm's risk tolerance and potentially minimize potential losses.